In [1]:
%load_ext autoreload
%autoreload 2 

### Check embedding error


In [12]:
path = "/mnt/ssd2t/mandy/Projects/few_shot_fcg/embeddings/x86_64_withVal_withPretrain_ghidra_6/word2vec/Advanced Micro Devices X86-64/aidra/81193e9a87778d7899a523adc7949f1a8af267d268e1dd51298165c22b890f4e.gpickle"

import networkx as nx
import pickle
import torch

with open(path, "rb") as f:
    G = pickle.load(f)
    
print(G)
# node = "0x4012f0L"
for node in G.nodes():
    print(G.nodes[node])
    if len(G.nodes[node]["x"]) == 0:
        G.nodes[node]["x"] = torch.zeros(128, dtype=torch.float32)
    if not isinstance(G.nodes[node]["x"], torch.Tensor):
        G.nodes[node]["x"] = torch.tensor(G.nodes[node]["x"], dtype=torch.float32)
# print(G.nodes[node])
# G.nodes[node]["x"] = []
# print(G.nodes[node])

# with open(path, "wb") as f:
#     pickle.dump(G, f)

MultiDiGraph named 'code' with 162 nodes and 248 edges
{'label': '"_init"', 'x': tensor([ 0.6937,  0.1734, -0.9755, -0.3928,  1.0027, -0.4447,  1.1724, -0.3646,
         0.0799,  0.0091, -0.8736, -1.0297,  0.5587,  0.4187,  0.2757, -1.0600,
         0.4051, -0.3973,  0.3419,  1.5886,  0.1039, -0.1193, -0.2372, -0.3832,
        -0.0600, -0.1604, -0.9059, -0.8325, -0.0312, -0.5448,  1.4323,  0.2972,
        -0.5172,  0.3059,  0.6481, -0.2236, -0.5616,  0.0966, -0.7813, -0.3143,
        -1.0249, -0.5717, -0.1713, -0.2384, -0.4751,  0.3269, -0.0434,  0.7471,
         0.6331, -0.3480,  0.0963, -0.3752, -1.0764,  0.1366, -0.1268, -0.5636,
        -0.2140,  1.0507,  0.1327,  0.1825,  0.5298,  0.7579, -0.6758,  0.2078,
        -0.4419,  0.1300,  0.0385,  0.5521,  0.1577,  0.8047,  0.2301, -1.4981,
         0.0862, -0.2380, -0.3474,  0.1513,  0.3114, -0.5933, -0.2390,  0.2813,
         1.0792,  0.2955, -0.1429, -0.6291,  0.5015,  1.1359,  1.0196, -0.5711,
        -0.3439,  0.2351,  0.4111, -0.3

In [13]:
from torch_geometric.utils import from_networkx
from torch_geometric.data import Data

data = from_networkx(G)
print(data)

Data(x=[162, 128], edge_index=[2, 248], label=[162], name='code')


In [ ]:
data = from_networkx(G, group_node_attrs=["x"])
print(data)

Data(edge_index=[2, 248], label=[162], name='code', x=[162, 128])


In [1]:
import pickle
path = "/mnt/ssd2t/mandy/Projects/few_shot_fcg/embeddings/x86_64_withVal_withPretrain_ghidra_888/word2vec/testData.pkl"

with open(path, "rb") as f:
    data = pickle.load(f)
print(data)

[[Data(x=[186, 128], edge_index=[2, 240], label=[186], name='code', y=0), Data(x=[160, 128], edge_index=[2, 220], label=[160], name='code', y=0), Data(x=[159, 128], edge_index=[2, 216], label=[159], name='code', y=0), Data(x=[211, 128], edge_index=[2, 258], label=[211], name='code', y=0), Data(x=[160, 128], edge_index=[2, 219], label=[160], name='code', y=0), Data(x=[172, 128], edge_index=[2, 270], label=[172], name='code', y=0), Data(x=[152, 128], edge_index=[2, 205], label=[152], name='code', y=0), Data(x=[139, 128], edge_index=[2, 213], label=[139], name='code', y=0), Data(x=[145, 128], edge_index=[2, 199], label=[145], name='code', y=0), Data(x=[148, 128], edge_index=[2, 202], label=[148], name='code', y=0), Data(x=[161, 128], edge_index=[2, 218], label=[161], name='code', y=0), Data(x=[150, 128], edge_index=[2, 207], label=[150], name='code', y=0), Data(x=[165, 128], edge_index=[2, 261], label=[165], name='code', y=0), Data(x=[155, 128], edge_index=[2, 277], label=[155], name='cod

/home/mandy/Tools/anaconda3/envs/prototype/lib/python3.11/site-packages/torch/storage.py:414: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(io.BytesIO(b))


In [6]:
import torch
def check_model_weights(model_path, device):
    # 載入模型
    checkpoint = torch.load(model_path, map_location=device)
    # 1. 檢查state dict是否為空
    if not checkpoint["model_state_dict"]:
        print("Warning: Model state dict is empty!")
        return False
        
    # 2. 印出模型的所有權重名稱和形狀
    for name, param in checkpoint["model_state_dict"].items():
        print(f"Layer: {name} | Shape: {param.shape}")
        print(param)
        
    # 3. 檢查權重是否包含非零值
    for param in checkpoint["model_state_dict"].values():
        if torch.all(param == 0):
            print(f"Warning: Found all-zero parameter tensor!")
            
    return True

In [7]:
path = "pretrained/x86_pretrained_20241122_1616/epoch_1342_best_backbone.pth"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

check_model_weights(path, device)

Layer: sage_convs.0.lin_l.weight | Shape: torch.Size([64, 128])
tensor([[-0.0105, -0.0067, -0.0859,  ...,  0.1475,  0.2736,  0.0953],
        [ 0.1123, -0.0609,  0.0241,  ...,  0.0036, -0.2800, -0.0800],
        [ 0.0679,  0.2026,  0.0164,  ..., -0.2146,  0.1522,  0.1322],
        ...,
        [-0.0825,  0.1536, -0.0767,  ..., -0.1602, -0.0237,  0.1719],
        [ 0.1604, -0.1947,  0.1364,  ...,  0.0306,  0.2789,  0.0696],
        [-0.1160, -0.1161, -0.1703,  ...,  0.0047,  0.0426,  0.1916]],
       device='cuda:0')
Layer: sage_convs.0.lin_l.bias | Shape: torch.Size([64])
tensor([-0.0476,  0.0360, -0.0779, -0.0628, -0.0565,  0.0955,  0.0905,  0.0331,
         0.0359, -0.0168, -0.0716, -0.0012,  0.0021,  0.0463,  0.0380, -0.0742,
        -0.0239,  0.0225,  0.0525, -0.0608,  0.0841,  0.0333,  0.0923, -0.0030,
        -0.0336,  0.0787, -0.0670, -0.0451,  0.0369, -0.0534, -0.0758, -0.0759,
         0.0346,  0.0072, -0.0788, -0.0713, -0.0141, -0.0510,  0.0316,  0.0117,
        -0.0101, -0.0

/tmp/ipykernel_818314/4270152637.py:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(model_path, map_location=device)


True

### Check validation dataset & test dataset difference

In [2]:
import pickle
testPath = "/mnt/ssd2t/mandy/Projects/few_shot_fcg/embeddings/x86_64_withVal_withPretrain_ghidra/word2vec/testData.pkl"
valPath = "/mnt/ssd2t/mandy/Projects/few_shot_fcg/embeddings/x86_64_withVal_withPretrain_ghidra/word2vec/valData.pkl"

with open(testPath, "rb") as f:
    test = pickle.load(f)
    
with open(valPath, "rb") as f:
    val = pickle.load(f)



In [ ]:
print(len(test[0]))
print(len(val[0]))

200
200


In [15]:
testAvgLen = sum([len(graph.x) for graph in test[0]]) / len(test[0])
valAvgLen = sum([len(graph.x) for graph in val[0]]) / len(val[0])

print(testAvgLen)
print(valAvgLen)

2366.12
1093.515


### Get label dictionary data when loading data

In [3]:
from utils import load_config
from loadDataset import LoadDataset
from trainModule import TestModule
import os

seeds = [6, 7, 10, 11, 19, 22, 31, 42, 666, 888]

for seed in seeds:
    configPath = f"/home/manying/Projects/fcgFewShot/config/config_NICT_Ghidra_x86_64_{seed}.json"
    options = load_config(configPath)
    
    options["paths"]["data"]["embedding_folder"] = "/home/manying/Projects/fcgFewShot/embeddings"
    dataset = LoadDataset(options)
    test = TestModule(configPath, dataset, options)


Loading all datasets...
train dataset shape: (1180, 16)
train dataset family number: 59
test dataset shape: (200, 16)
test dataset family number: 10
val dataset shape: (200, 16)
val dataset family number: 10
Setting up the testing module...
Loading data from /home/manying/Projects/fcgFewShot/embeddings/x86_64_withVal_withPretrain_ghidra_6/word2vec...
Loading testing data...


100%|██████████| 200/200 [00:00<00:00, 9316.95it/s]


Finish setting up the testing module
Loading all datasets...
train dataset shape: (1180, 16)
train dataset family number: 59
test dataset shape: (200, 16)
test dataset family number: 10
val dataset shape: (200, 16)
val dataset family number: 10
Setting up the testing module...
Loading data from /home/manying/Projects/fcgFewShot/embeddings/x86_64_withVal_withPretrain_ghidra_7/word2vec...
Loading testing data...


100%|██████████| 200/200 [00:00<00:00, 15074.14it/s]

Finish setting up the testing module
Loading all datasets...
train dataset shape: (1180, 16)
train dataset family number: 59
test dataset shape: (200, 16)
test dataset family number: 10
val dataset shape: (200, 16)
val dataset family number: 10
Setting up the testing module...
Loading data from /home/manying/Projects/fcgFewShot/embeddings/x86_64_withVal_withPretrain_ghidra_10/word2vec...
Loading testing data...


...

Finish setting up the testing module
Loading all datasets...
train dataset shape: (1180, 16)
train dataset family number: 59
test dataset shape: (200, 16)
test dataset family number: 10
val dataset shape: (200, 16)
val dataset family number: 10
Setting up the testing module...
Loading data from /home/manying/Projects/fcgFewShot/embeddings/x86_64_withVal_withPretrain_ghidra_666/word2vec...
Loading testing data...


100%|██████████| 200/200 [00:00<00:00, 29408.95it/s]


Finish setting up the testing module
Loading all datasets...
train dataset shape: (1180, 16)
train dataset family number: 59
test dataset shape: (200, 16)
test dataset family number: 10
val dataset shape: (200, 16)
val dataset family number: 10
Setting up the testing module...
Loading data from /home/manying/Projects/fcgFewShot/embeddings/x86_64_withVal_withPretrain_ghidra_888/word2vec...
Loading testing data...


100%|██████████| 200/200 [00:00<00:00, 27149.36it/s]

Finish setting up the testing module


### Test: train on 10-way test on 5-way performance
Testing on seed_6_baseline: LP 10-way 5-shot

In [1]:
from utils import load_config, save_config
import os
from loadDataset import LoadDataset
from trainModule import TestModule
configPath = "/home/mandy/Projects/few_shot_fcg/checkpoints/x86_64_withVal_withPretrain_ghidra_6_baseline/10way_5shot_LabelPropagation_alpha0.7_k20_20250315_155140/config.json"
options = load_config(configPath)
newConfigPath = os.path.join(os.path.dirname(configPath), "config_5way.json")
### change settings
options["settings"]["few_shot"]["test"]["class_per_iter"] = 5
options["settings"]["train"]["distance"] = "euclidean"
save_config(options, newConfigPath)

dataset = LoadDataset(options)

test = TestModule(newConfigPath, dataset)
test.eval()

/home/mandy/Projects/few_shot_fcg/trainModule.py:397: SyntaxWarning: assertion is always true, perhaps remove parentheses?
  assert(mode == "closedset" or mode == "openset", "Mode not supported")
/home/mandy/Projects/few_shot_fcg/trainModule.py:407: SyntaxWarning: assertion is always true, perhaps remove parentheses?
  assert(self.loss_fn.enable_openset == False, "Open set evaluation is not supported in this mode")
/home/mandy/Projects/few_shot_fcg/trainModule.py:415: SyntaxWarning: assertion is always true, perhaps remove parentheses?
  assert(self.loss_fn.enable_openset == True, "You need to enable openset in the model")
/home/mandy/Projects/few_shot_fcg/trainModule.py:416: SyntaxWarning: assertion is always true, perhaps remove parentheses?
  assert(self.opensetTestLoader is not None, "You need to load openset data")


Loading all datasets...
train dataset shape: (1180, 16)
train dataset family number: 59
test dataset shape: (200, 16)
test dataset family number: 10
val dataset shape: (200, 16)
val dataset family number: 10
Setting up the testing module...
Loading data from /mnt/ssd2t/mandy/Projects/few_shot_fcg/embeddings/x86_64_withVal_withPretrain_ghidra_6/word2vec/testData.pkl...
Loading data from /mnt/ssd2t/mandy/Projects/few_shot_fcg/embeddings/x86_64_withVal_withPretrain_ghidra_6/word2vec...
Generating closed set testing data...
Loading testing data...
Finish setting up the testing module
Model path is not provided. Using the best model...
Record evaluation log...
Loading model from /home/mandy/Projects/few_shot_fcg/checkpoints/x86_64_withVal_withPretrain_ghidra_6_baseline/10way_5shot_LabelPropagation_alpha0.7_k20_20250315_155140/epoch_10_0.9036666864156723_best.pth


/home/mandy/Tools/anaconda3/envs/prototype/lib/python3.11/site-packages/torch/storage.py:414: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(io.BytesIO(b))


Model: LabelPropagation(
  (encoder): GraphSAGELayer(
    (sage_convs): ModuleList(
      (0-2): 3 x SAGEConv(128, 128, aggr=mean)
    )
    (norms): ModuleList(
      (0-2): 3 x BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
  )
  (relation): GraphRelationNetwork(
    (sage): GraphSAGELayer(
      (sage_convs): ModuleList(
        (0): SAGEConv(128, 64, aggr=mean)
        (1): SAGEConv(64, 32, aggr=mean)
      )
      (norms): ModuleList(
        (0): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (1): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (fc): Sequential(
      (0): Linear(in_features=32, out_features=16, bias=True)
      (1): ReLU()
      (2): Linear(in_features=16, out_features=1, bias=True)
    )
  )
)
Start evaluation... (testing dataset / closedset dataset)
Epoch 1


Testing: 100%|██████████| 100/100 [00:06<00:00, 16.33it/s]


...

Epoch 9


Testing: 100%|██████████| 100/100 [00:06<00:00, 16.05it/s]


Epoch 10


Testing: 100%|██████████| 100/100 [00:06<00:00, 16.23it/s]

Testing accuracy: 0.8069
Closed set test accuracy: 0.8069333525896072
Finish evaluation


### Analysis diff backbone parameters

In [4]:
from models import GraphSAGELayer, GATLayer, GCNLayer, GINLayer



def count_parameters(model):
    """計算模型的可訓練參數總數"""
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

# 對各個模型進行統計
def compare_model_parameters():
    # 設定一致的維度參數，確保公平比較
    input_dim = 128
    hidden_dim = 128
    output_dim = 128
    num_layers = 3
    
    # 實例化各種模型
    sage_model = GraphSAGELayer(input_dim, hidden_dim, output_dim, num_layers)
    gat_model = GATLayer(input_dim, hidden_dim, output_dim, num_layers, heads=4)
    gcn_model = GCNLayer(input_dim, hidden_dim, output_dim, num_layers)
    gin_model = GINLayer(input_dim, hidden_dim, output_dim, num_layers)
    
    # 計算參數數量
    sage_params = count_parameters(sage_model)
    gat_params = count_parameters(gat_model)
    gcn_params = count_parameters(gcn_model)
    gin_params = count_parameters(gin_model)
    
    # 輸出結果
    print(f"{'Model':<12} {'Parameters':<12} {'Ratio to GraphSAGE':<20}")
    print("-" * 44)
    print(f"{'GraphSAGE':<12} {sage_params:<12,d} {1.0:<20.2f}")
    print(f"{'GAT':<12} {gat_params:<12,d} {gat_params/sage_params:<20.2f}")
    print(f"{'GCN':<12} {gcn_params:<12,d} {gcn_params/sage_params:<20.2f}")
    print(f"{'GIN':<12} {gin_params:<12,d} {gin_params/sage_params:<20.2f}")
    
    return {
        "GraphSAGE": sage_params,
        "GAT": gat_params,
        "GCN": gcn_params,
        "GIN": gin_params
    }

# 調用函數
model_params = compare_model_parameters()

Model        Parameters   Ratio to GraphSAGE  
--------------------------------------------
GraphSAGE    99,456       1.00                
GAT          51,072       0.51                
GCN          50,304       0.51                
GIN          100,611      1.01                
